In [1]:
import pandas as pd  # 载入数据包
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
plt.rcParams['font.sans-serif'] = ['SimHei']  # 用于显示中文
plt.rcParams['axes.unicode_minus'] = False

## 数据预处理
使用前先将源数据内的缺失值进行填充，并剔除几列用不到的数据，处理完毕的数据会保存为'./data_pca.csv'文件。

In [3]:
# read
train_data_0 = pd.read_csv('train_titanic.csv', index_col='PassengerId')  # 读取并查看原始文件信息

In [4]:
# copy
train_data = train_data_0.copy()  # 备份数据
# replace
train_data.Sex = train_data.Sex.replace({'female': 0, 'male': 1})  # 对Sex性别列进行处理男性为1，女性为0
train_data.Embarked = train_data.Embarked.replace({'C': 0, 'Q': 1, 'S': 2})  # 对登船口岸列进行处理
# fillna
train_data.Age.fillna(train_data.Age.mean(), inplace=True)  # 用Age列的平均值对这一列的空缺值进行填充
train_data.Embarked.fillna(method='ffill', inplace=True)  # Embarked列中空缺值的填充使用空缺值前一项进行
# drop
train_data.drop(columns=['Survived', 'Name', 'Ticket', 'Cabin'], inplace=True)
# save
train_data.to_csv('data_pca.csv')  # 保存处理后的数据

## 正式开始
泰坦尼克号数据结构分析：
- Pclass:乘客所持票类，有三种值(1：lower,2：middle,3：upper)
- Survived:0代表死亡，1代表存活
- Name:乘客姓名
- Sex:乘客性别(男：1，女：0)
- Age:乘客年龄(缺失值取平均值)
- SibSp:乘客兄弟姐妹/配偶的个数(整数值)
- Parch:乘客父母/孩子的个数(整数值)
- Ticket:票号(字符串)
- Fare:乘客所持票的价格(浮点数，0-500不等)
- Cabin:乘客所在船舱
- Embark:乘客登船港口:S、C、Q

### 读取数据

In [5]:
# read
data = pd.read_csv('./data_pca.csv', index_col='PassengerId')  # 读取预处理后保存的csv数据，并设置PassengerId为索引值
data.head(3)

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
PassengerId,,,,,,,
1,3,1,22.0,1,0,7.2500,2.0
2,1,0,38.0,1,0,71.2833,0.0
3,3,0,26.0,0,0,7.9250,2.0


In [6]:
data[data['Fare'] == data['Fare'].max()]  # Pclass对应船舱等级，可以看到票价（Fire）最高的三位乘客都是头等舱（Pclass=1）

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
PassengerId,,,,,,,
259,1,0,35.0,0,0,512.3292,0.0
680,1,1,36.0,0,1,512.3292,0.0
738,1,1,35.0,0,0,512.3292,0.0


### 数据中心化与标准化

这一步是主成分分析（PCA）的数据预处理，首先计算出矩阵中每一列的平均值，然后让每一列中的每一个元素都减去该列的平均值。在进行完这一步之后可以观察每一列数据的分散情况，如果列与列之间的差距仍过大，可以再计算出每一列的方差，然后再将减去平均值后的结果除以每一列的方差（这一步是可选步骤，叫做方差归一化）。

如果两步都做了，这个叫做数据的normalization——对一个非标准的正态分布随机变量转化为标准的正态分布随机变量。

In [7]:
X = (data - data.mean()) / data.std(ddof=0)  # 获取经过Normalization的矩阵X

这里一个注意点：
我们使用ddof=1样本标准偏差或ddof=0总体标准偏差作为参数来控制。

In [8]:
# pd.Series([7,20,22,22]).std()==np.std([7,20,22,22])
# pd.Series([7,20,22,22]).std(ddof=0)==np.std([7,20,22,22])

### 预检验
首先进行KMO和Bartlett的检验，判断是否可以进行主成分分析。 对于KMO值：0.8上非常合适做主成分分析，0.7-0.8之间一般适合，0.6-0.7之间不太适合，0.5-0.6之间表示差，0.5下表示极不适合，对于 Bartlett的检验（p < 0.05,严格来说p < 0.01），若显著性小于0.05或0.01，拒绝原假设，则说明可以做主成分分析，若不拒绝原假设，则说明这些变量可能独立提供一些信息，不适合做主成分分析；

In [9]:
from factor_analyzer.factor_analyzer import calculate_bartlett_sphericity

# 巴特利特P值小于0.01，KMO值大于0.6；说明此数据适合做因子分析。
chi_square_value, p_value = calculate_bartlett_sphericity(X)  # 计算巴特利特P值
p_value

2.1502836817511034e-177

In [10]:
from factor_analyzer.factor_analyzer import calculate_kmo

kmo_all, kmo_model = calculate_kmo(X)
kmo_model

0.5800403597237765

In [11]:
# 查看数据各列之间的相关性
matrix = X.corr()  # 获取各列之间的相关性，这个matrix其实就是X的协方差矩阵
mask = np.triu(np.ones_like(matrix, dtype=bool))  # mask掉上三角部分
get_ipython().run_line_magic('matplotlib', 'notebook')  # 俺也不知道这句代码有啥用，但是加了可以让Jupyter中的图更加漂亮
plt.figure(figsize=(8, 5), dpi=120)
cmap = sns.diverging_palette(370, 120, n=80, as_cmap=True)
sns.heatmap(matrix, mask=mask, center=0, cmap=cmap, annot=True, square=True, fmt='.2f', vmax=1, vmin=-1)
plt.title('泰坦尼克号数据集各列之间的相关性')

<IPython.core.display.Javascript object>

Text(0.5, 1.0, '泰坦尼克号数据集各列之间的相关性')

### 算法降维处理
在选取特征值的时候，要选取比较大的，特征值大的话数据方差就大，这也就意味着在经过降维后，可以最大程度上保留数据的差异性。

降维的过程实际就是寻找投影方向的过程。

从一个方面讲:

- 好的投影方向，可以使得投影误差最小化。
- 投影误差是各个数据点到投影点距离，这也就意味着经过降维处理后的损失达到最小。

从另一个方面讲:

- 好的投影方向，也就意味着在该方向投影的方差应该最大(分散度/区分度高)，可是使得信息在最大程度上得到保留。
- 复原误差小、重建容易。

#### 降维的步骤

目标：将n维数据降低为k维(k<n)。
（1）计算协方差矩阵：协方差矩阵（n×n）其实就是原数据矩阵中每两列之间的相关性系数；
（2）计算投影方向：对E进行奇异值分解(Singular Value Decomposition) [U,S,V] = svd (SIGMA)

其中U是一个nxn的矩阵，它每列是SIGMA的一个特征向量(线性代数概念)，U的前k列就是PCA的前k个投影向;我们将U的前k列记为投影矩阵P.
U的每一列都是一个特征向量，因此它的模长都为1，且特征向量之间都是正交的，乘积都为0

计算投影：

- 上一步我们得到了投影矩阵$P \in R^{n \times k}$；
- 对于第i个n维样本$x^{(i)}$，计算$z^{(i)} = x^{(i)} * P$，$z^{(i)}$就是降维后的第i个样本。

$$
X \in R^{m \times n} \times P \in R^{n \times k} = Z \in R^{m \times k}
$$


这里用最简单的方式实现一个PCA分析，下方会进行详细介绍。

In [12]:
# SIGMA  # 各列之间的相关性（可以查看上面使用X.corr()命令获取矩阵X中各列之间的相关性所绘制的热图中的数据其实就是计算了矩阵X的协方差）
SIGMA = (X.T @ X) / X.shape[0]  # 计算协方差矩阵（代表原矩阵中各列之间的相关性），@用来计算矩阵之间的乘法，.shape[0]获取矩阵中有多少行
# 奇异值分解
U, S, V = np.linalg.svd(SIGMA)  # S 是个对角矩阵，是特征值，U是特征值对应的标准正交特征向量
# U
P = U[:, :2]  # 这里取U的前两列就是前两个投影的方向
# P
Z = X @ P  # Z就是经过降维后的矩阵
# Z

In [42]:
X = (data - data.mean()) / data.std(ddof=0)  # 获取经过Normalization的矩阵X
SIGMA = (X.T @ X) / X.shape[0]

SIGMA

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
Pclass,1.000000,0.131900,-0.331339,0.083081,0.018443,-0.549500,0.168430
Sex,0.131900,1.000000,0.084153,-0.114631,-0.245489,-0.182333,0.113807
Age,-0.331339,0.084153,1.000000,-0.232625,-0.179191,0.091566,-0.032025
SibSp,0.083081,-0.114631,-0.232625,1.000000,0.414838,0.159651,0.070111
Parch,0.018443,-0.245489,-0.179191,0.414838,1.000000,0.216225,0.041732
Fare,-0.549500,-0.182333,0.091566,0.159651,0.216225,1.000000,-0.228364
Embarked,0.168430,0.113807,-0.032025,0.070111,0.041732,-0.228364,1.000000


In [37]:
X.corr()

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
Pclass,1.000000,0.131900,-0.331339,0.083081,0.018443,-0.549500,0.168430
Sex,0.131900,1.000000,0.084153,-0.114631,-0.245489,-0.182333,0.113807
Age,-0.331339,0.084153,1.000000,-0.232625,-0.179191,0.091566,-0.032025
SibSp,0.083081,-0.114631,-0.232625,1.000000,0.414838,0.159651,0.070111
Parch,0.018443,-0.245489,-0.179191,0.414838,1.000000,0.216225,0.041732
Fare,-0.549500,-0.182333,0.091566,0.159651,0.216225,1.000000,-0.228364
Embarked,0.168430,0.113807,-0.032025,0.070111,0.041732,-0.228364,1.000000


In [13]:
# V
# ew, ev = np.linalg.eig(X.T.dot(X))
ew, ev = np.linalg.eig(np.cov(X.T))  # ew: 特征值，ev: 特征向量
ew_order = np.argsort(ew)[::-1]  # 将ew按照从大到小的顺序排列
ew_sort = ew[ew_order]  # 特征根，主要是看主成分对于变量解释的贡献率（可以理解为究竟需要多少主成分才能把变量表达为100%），一般都要表达到90%以上才可以，否则就要调整因子数据。
ev_sort = ev[:, ew_order]
# print(ew_sort)

### 方差解释表格
**图表说明:**
上表为总方差解释表格，主要是看主成分对于变量解释的贡献率（可以理解为究竟需要多少主成分才能把变量表达为100%），一般都要表达到90%以上才可以，否则就要调整因子数据。一般情况下,方差解释率越高，说明该主成分越重要，权重占比也应该越高。

方差解释表中，在主成分4时，总方差解释的特征根低于1，变量解释的贡献率达到74.899%，以上仅为参考。
若特征根小于1临界值过大，也可以集合具体情况具体分析。

In [14]:
# 输出一个方差解释表格
var_inter_0 = pd.DataFrame({'特征值': ew_sort})  # 添加特征值列
var_inter_0['百分比'] = (var_inter_0 / var_inter_0.sum())  # 添加百分比列
var_inter_0['累计百分比'] = var_inter_0['百分比'].cumsum()  # 计算累计百分比
var_inter = var_inter_0.copy()
var_inter[['百分比', '累计百分比']] = var_inter[['百分比', '累计百分比']].applymap(lambda x: format(x, '.1%'))  # 保留一位小数
var_inter_0['1'] = 1  # 前两列特征值大于1，且51%以上可以用来解释原来7个变量，因此取前两列
var_inter_0


,特征值,百分比,累计百分比,1
0,1.859829,0.265392,0.265392,1
1,1.720213,0.245469,0.510861,1
2,0.988894,0.141112,0.651973,1
3,0.838403,0.119637,0.771610,1
4,0.673828,0.096153,0.867763,1
5,0.558841,0.079745,0.947508,1
6,0.367857,0.052492,1.000000,1


In [15]:
pd.DataFrame(ew_sort).plot(kind='bar').legend(('特征值',))  # 添加图例必须多一个逗号，否则只显示第一个字符
plt.xticks(rotation=0)
# print(ev_sort)

<IPython.core.display.Javascript object>

(array([0, 1, 2, 3, 4, 5, 6]),
 [Text(0, 0, '0'),
  Text(1, 0, '1'),
  Text(2, 0, '2'),
  Text(3, 0, '3'),
  Text(4, 0, '4'),
  Text(5, 0, '5'),
  Text(6, 0, '6')])

In [16]:
# ev_sort[:, :2]  # 前两列就很大程度上可以用来解释原来7个变量，因此取前两列# V
V = ev_sort[:, :2]  # 前两列就很大程度上可以用来解释原来7个变量，因此取前两列

In [17]:
# 构建一个因子数据表
factor_load_table = pd.DataFrame(V)  # 因子载荷数据表，可以分析到每个主成分中隐变量的重要性。
factor_load_table.index = train_data.columns
factor_load_table.columns = ['PCA 0', 'PCA 1']
factor_load_table.index.name = 'var'
factor_load_table['公共度'] = factor_load_table['PCA 0'] ** 2 + factor_load_table[
    'PCA 1'] ** 2  # 公因子方差：每个变量对应的PCA0与PCA1的平方和
factor_load_table = factor_load_table
factor_load_table

,PCA 0,PCA 1,公共度
var,,,
Pclass,0.530014,0.343629,0.398995
Sex,0.346116,-0.217909,0.167281
Age,-0.134322,-0.500377,0.268420
SibSp,-0.190450,0.538409,0.326155
Parch,-0.292306,0.513704,0.349335
Fare,-0.613415,-0.050032,0.378781
Embarked,0.288540,0.166667,0.111033


In [18]:
# U的每一列都是一个特征向量，因此它的模长都为1，且特征向量之间都是正交的，乘积都为0
# (factor_load_table['PCA 0'] * factor_load_table['PCA 1']).sum()  # 两个特征向量的乘积应为0
# (factor_load_table['PCA 0']**2).sum()  # 每一个特征向量的模长都为0
# (factor_load_table['PCA 1']**2).sum()

In [19]:
plt.figure(figsize=(8, 5), dpi=120)
sns.heatmap(factor_load_table)

<IPython.core.display.Javascript object>

<AxesSubplot:ylabel='var'>

### 绘图

In [20]:
# X_new
X_new = X.dot(V)
X_new['Survived'] = train_data_0['Survived']  # 将生存数据添加到DATaFrame中，用于绘制散点图的时候可以区分死亡与生存的区别
X_new

,0,1,Survived
PassengerId,,,
1,1.307635,0.533029,0
2,-2.369777,-0.934450,1
3,0.706240,0.345947,1
4,-1.386124,-0.380068,1
5,1.336180,-0.456877,0
...,...,...,...
887,0.723446,-0.565049,0
888,-0.762629,-0.229217,1
889,-0.422228,1.951713,0


In [21]:
plt.figure(figsize=(8, 5), dpi=120)  # 设置绘图尺寸
sns.scatterplot(x=factor_load_table['PCA 0'], y=factor_load_table['PCA 1'], hue=factor_load_table.index)

<IPython.core.display.Javascript object>

<AxesSubplot:xlabel='PCA 0', ylabel='PCA 1'>

In [22]:
# scatter
plt.figure(figsize=(8, 5), dpi=120)  # 设置绘图尺寸
sc_survived = plt.scatter(X_new[X_new['Survived'] == 1].iloc[:, 0], X_new[X_new['Survived'] == 1].iloc[:, 1], s=5,
                          c='blue',
                          cmap=plt.cm.coolwarm)  # 提取生存的
sc_un_survived = plt.scatter(X_new[X_new['Survived'] == 0].iloc[:, 0], X_new[X_new['Survived'] == 0].iloc[:, 1], s=5,
                             c='red',
                             cmap=plt.cm.coolwarm)  # c=train_X0.Survived设置颜色反应生还状态，是死是活
# plt.grid(False)  # 是否显示网格线
plt.xlabel('PCA 0')  # 设置X、Y轴坐标名称
plt.ylabel('PCA 1')
plt.legend(handles=[sc_survived, sc_un_survived], labels=['生存', '死亡'], loc='upper left')

<IPython.core.display.Javascript object>

In [23]:
# 年龄分布直方图
# get_ipython().run_line_magic('matplotlib', 'notebook')
plt.figure(figsize=(8, 5), dpi=120)  # 设置绘图尺寸
train_data_0.Age.hist()

<IPython.core.display.Javascript object>

<AxesSubplot:>

In [24]:
# 成分矩阵表
composition_matrix = factor_load_table.loc[:, ['PCA 0', 'PCA 1']]  # 成分矩阵表 = 因子载荷数据表/对应每一个特征值
composition_matrix['PCA 0'] = composition_matrix['PCA 0'] / ew_sort[0]  # 这里有个问题，有的软件里除的是根号下特征值，有的没有
composition_matrix['PCA 1'] = composition_matrix['PCA 1'] / ew_sort[1]
composition_matrix.round(3)

,PCA 0,PCA 1
var,,
Pclass,0.285,0.200
Sex,0.186,-0.127
Age,-0.072,-0.291
SibSp,-0.102,0.313
Parch,-0.157,0.299
Fare,-0.330,-0.029
Embarked,0.155,0.097


图表说明：
上表为成分矩阵表，意在说明各个成分的所包含的因子得分系数（主成分载荷），用于计算出成分得分，得出因子公式，其计算公式为：线性组合系数*（方差解释率/累积方差解释率），最后将其归一化即为因子权重分。
线性组合系数，公式为：因子载荷系数除以对应特征根，即成分矩阵的系数。
智能分析：
模型的公式：
F1=-0.389×Pclass-0.254×Sex+0.099×Age+0.14×SibSp+0.214×Parch+0.45×Fare-0.212×Embarked
F2=0.262×Pclass-0.166×Sex-0.382×Age+0.411×SibSp+0.392×Parch-0.038×Fare+0.127×Embarked
由上可以得到：
F=(0.265/0.511)×F1+(0.245/0.511)×F2

In [25]:
# sns.pairplot(train_data_0)  # 各列的相关性图

In [26]:
# 因子权重分析表
factor_weight_analysis = pd.DataFrame(var_inter_0.loc[:1, ['百分比', '累计百分比']])
factor_weight_analysis['权重'] = factor_weight_analysis['百分比'] / factor_weight_analysis['百分比'].sum()
factor_weight_analysis.index = ['PCA 0', 'PCA 1']
factor_weight_analysis.index.name = '主成分'
(factor_weight_analysis * 100).round(1).astype(str) + '%'  # 以保留一位小数、百分比的形式显示

,百分比,累计百分比,权重
主成分,,,
PCA 0,26.5%,26.5%,51.9%
PCA 1,24.5%,51.1%,48.1%


In [27]:
# 综合得分表（其实就是X_new中的数据）
comprehensive_score = pd.DataFrame(X_new.iloc[:, [0, 1]])
comprehensive_score.columns = ['PCA 0', 'PCA 1']
comprehensive_score['综合得分'] = comprehensive_score.iloc[:, 0] * factor_weight_analysis.loc[
    'PCA 0', '百分比'] + comprehensive_score.iloc[:, 1] * factor_weight_analysis.loc['PCA 1', '百分比']
comprehensive_score.sort_values(by='综合得分', ascending=False)  # 按照综合得分列倒序排列

,PCA 0,PCA 1,综合得分
PassengerId,,,
387,-0.381898,4.531119,1.010896
262,-0.038176,3.981215,0.967133
851,-0.047123,3.942822,0.955334
481,-0.464592,4.223070,0.913333
165,0.242825,3.412203,0.902034
...,...,...,...
381,-4.168049,-1.734389,-1.531904
300,-4.860549,-1.424920,-1.639723
680,-7.261947,-1.608737,-2.322155


In [28]:
data.loc[comprehensive_score.sort_values(by='综合得分', ascending=False).head(5).index]  # 获取综合得分前五的人员

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
PassengerId,,,,,,,
387,3,1,1.0,5,2,46.9000,2.0
262,3,1,3.0,4,2,31.3875,2.0
851,3,1,4.0,4,2,31.2750,2.0
481,3,1,9.0,5,2,46.9000,2.0
165,3,1,1.0,4,1,39.6875,2.0


In [29]:
x = pd.DataFrame([[1, -2, 0], [-2, 5, 0], [0, 0, 0.2]])

# SIGMA = (x.T @ x) / (x.shape[0]-1)  # 计算协方差矩阵（代表原矩阵中各列之间的相关性），@用来计算矩阵之间的乘法，.shape[0]获取矩阵中有多少行
# SIGMA = x / (x.shape[0]-1)
# 奇异值分解
U, S, V = np.linalg.svd(x)  # S 是个对角矩阵，是特征值
# U
S
# P = U[:, :2]  # 这里取U的前两列就是前两个投影的方向
# P
# Z = X @ P  # Z就是经过降维后的矩阵

array([5.82842712, 0.2       , 0.17157288])